Task: Aggregate a Historical Dataset into Hourly Intervals

As a data science expert, you are to construct a new dataset aggregated by the hour from a historical dataset that records data every 5 minutes. The primary focus is on bike sharing stations, using the 'station_id' for alignment and calculating the 'percentage_docks_available' at these stations. Follow these detailed instructions to complete the task:

Merge Datasets Instructions:

1 - Combine the relevant datasets using the 'station_id' as the key.
Remove any records (registries) where 'station_id' does not have a match in both datasets. This ensures only data with complete 'station_id' references are considered.

2 - Aggregate Records:
Group the data by 'station_id' and the 'last_reported' timestamp. Include index, station_id, month, day, hour columns.

3 - Calculate Dock Availability:
Within the aggregation, compute the 'percentage_docks_available' for each group. This is done by dividing 'num_docks_available' by 'capacity' for each record and expressing the result as a percentage:

percentage_docks_available =(num_docks_available/capacity)

Please provide the code based on this datasets:

Dataset 1: station_id, num_bikes_available, num_bikes_available_types.mechanical, num_bikes_available_types.ebike, num_docks_available, is_installed, is_renting, is_returning, last_reported, is_charging_station, status, last_updated, ttl

Dataset 2: station_id, name, physical_configuration, lat, lon, altitude, address, post_code, capacity, is_charging_station, nearby_distance, _ride_code_support, rental_uris, cross_street

In [ ]:

def calculate_percentage(group):
    mean_docks = group['num_docks_available'].mean()
    mean_capacity = group['capacity'].mean()
    return (mean_docks / mean_capacity)

In [ ]:
from IPython.display import display
import dask.dataframe as dd
from tqdm.notebook import tqdm
import pandas as pd
import sys

sys.path.append('../src')
from data.load_raw_quarter import load_raw_quarter
from data.exploration import detect_anomalies

# Cargar la información sobre las estaciones
stations_df = pd.read_csv("../data/raw/Informacio_Estacions_Bicing.csv")
# Convertir station_id a set para búsquedas más rápidas
valid_station_ids = set(stations_df['station_id'])
stations_df.set_index('station_id', inplace=True)

# Cargar los archivos de datos brutos
files_df = load_raw_quarter()

# New DataFrame para registrar anomalías
new_df = pd.DataFrame()

for file_name, df in tqdm(files_df.items(), desc="Processing Files"):
    try:
        print(f'START {file_name}'.center(100, "="))

        if isinstance(df, dd.DataFrame):
            df = df.compute()
        
        ## Build DataFrame
        # Convert 'last_reported' to datetime
        new_df['last_reported'] = dd.to_datetime(new_df['last_reported'], unit='s')
        # Extract components needed for grouping
        new_df['year'] = new_df['last_reported'].dt.year
        new_df['month'] = new_df['last_reported'].dt.month
        new_df['day'] = new_df['last_reported'].dt.day
        new_df['hour'] = new_df['last_reported'].dt.hour
        
        grouped = new_df.groupby(['station_id', 'year', 'month', 'day', 'hour'])

        percentage_docks_available = grouped.apply(calculate_percentage, meta=('x', 'f8')).reset_index()
        percentage_docks_available.columns = ['station_id', 'year', 'month', 'day', 'hour', 'percentage_docks_available']
        
        result = percentage_docks_available.compute()
        result.to_csv('hourly_aggregated_data.csv', index=False)

        print(f'END {file_name}'.center(100, "="))
        
    except Exception as e:
        print(f"Error processing {file_name}: {e}")
